In [1]:
import pandas as pd

df_ratings = pd.read_csv("./ml-1m/ratings.dat",sep="::",names=['user_id','movie_id','rating','timestamp'])
df_movies = pd.read_csv("./ml-1m/movies.dat",sep="::",names=['movie_id','title','genre'])
df_users = pd.read_csv("./ml-1m/users.dat",sep="::",names=['user_id','gender','age','occupation', 'zip_code'])

/home/fm-pc-lt-95/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/fm-pc-lt-95/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
/home/fm-pc-lt-95/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are

In [2]:
movie_rating_df = pd.merge(df_ratings, df_movies, on='movie_id')
movie_user_rating_df = pd.merge(movie_rating_df, df_users, on= 'user_id')
# movie_user_rating_df = movie_user_rating_df.iloc[:100]
movie_user_rating_df.head()

,user_id,movie_id,rating,timestamp,title,genre,gender,age,occupation,zip_code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [3]:
import numpy as np

In [4]:
msk = np.random.rand(len(movie_user_rating_df)) < 0.7

In [5]:
train = movie_user_rating_df[msk]
test = movie_user_rating_df[~msk]
print(len(train))
print(len(test))

79
21


## Encode the fields

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
unique_feature_num = 0
total_feature = 0

In [8]:
def encode(feature_index):
    a=np.zeros(unique_feature_num, dtype=int)
    a[feature_index]=1
    return a

In [9]:
features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip_code']
label = 'rating'

In [10]:
encoded_df = pd.DataFrame()
for feature in features:
#     Change the columns to str
    train[feature] = train[feature].astype(str)
    test[feature] = test[feature].astype(str)
    
    lbe = LabelEncoder()
    lbe.fit(train[feature])
    lbe.classes_ = np.append(lbe.classes_,'others')
    unique_feature_num = len(lbe.classes_)
    total_feature += unique_feature_num
    np.save(f"./{feature}.npy", lbe.classes_)
    
    train[feature] = lbe.transform(train[feature])
    train[feature] = train[feature].apply(encode)
    
#     Assign others if not present in classes
#     import pdb; pdb.set_trace()
    presence_mask = test[feature].isin(lbe.classes_.tolist())
    test[feature][~presence_mask]='others'
    
#     Encode test
    test[feature] = lbe.transform(test[feature])
    test[feature] = test[feature].apply(encode)


/home/fm-pc-lt-95/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/fm-pc-lt-95/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/fm-pc-lt-95/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [11]:
train[[label, *features]].to_csv("./train.csv")
test[[label, *features]].to_csv("./test.csv")

In [12]:
train.iloc[0]['movie_id']

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Define Dataset

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
from copy import deepcopy

In [14]:
input_data = []
input_data.extend([1,2])

In [15]:
class Movie_Dataset(Dataset):
    def __init__(self, df, features, label):
        self.df = deepcopy(df)
        self.features = features
        self.label = label
#         print(self.df.head())
    
    def __len__(self):
        return(len(self.df))
    
    def __getitem__(self, index):
        input_data = []
        row_data = self.df.iloc[index]
        for feature in self.features:
            input_data.extend(row_data[feature])
#         import pdb; pdb.set_trace()
        input_data = np.array(input_data, dtype=float)
        label = row_data[self.label]
        label = np.array([label])
#         import pdb; pdb.set_trace()
        
        return { 'input': torch.from_numpy(input_data).type(torch.FloatTensor), 'label': torch.from_numpy(label).type(torch.FloatTensor)}
        

In [16]:
train_dataset = Movie_Dataset(train, features, label)
test_dataset = Movie_Dataset(test, features, label)

In [17]:
train_dataset.df.iloc[0].user_id

array([1, 0, 0])

In [18]:
train_dataset[1]

{'input': tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
         1., 0., 0.]), 'label': tensor([3.])}

## Define DataLoader 

In [19]:
train_data_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=16)

## Define Linear Regression Model

In [20]:
import torch.nn as nn
import torch.nn.functional as F

In [21]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_num):
        super().__init__()
        self.linear_layer = nn.Linear(in_features=input_num, out_features=1)
    
    def forward(self, data):
        rating = self.linear_layer(data)
        return rating

In [22]:
model = LinearRegressionModel(total_feature)

## Define a loss function and optimizer

In [23]:
import torch.optim as optim

In [24]:
loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

## Train

In [25]:
epoch_num = 100
loss_lst = []

for num in range(epoch_num):    
    for batch_number, batch in enumerate(train_data_loader):
        input = batch['input']
        label = batch['label']

        model.train()
        output = model(input)

        model.zero_grad()

        loss = loss_function(output, label)
        loss.backward()

        optimizer.step()
        loss_lst.append(loss.item())
        
    print(num, np.mean(loss_lst))
    

0 17.48626537322998
1 16.91332483291626
2 16.38647238413493
3 15.86154384613037
4 15.36611026763916
5 14.886230691274006
6 14.429491806030274
7 13.992306685447693
8 13.575195439656575
9 13.178629436492919
10 12.797926365245472
11 12.431835333506266
12 12.08192631648137
13 11.747512435913086
14 11.427915547688801
15 11.121054977178574
16 10.826703576480641
17 10.544350094265408
18 10.272328959013286
19 10.011821377277375
20 9.762727925890967
21 9.522946006601506
22 9.29419436869414
23 9.073626073201497
24 8.861455341339111
25 8.657499582950885
26 8.461876616654573
27 8.273433150563921
28 8.092239518001161
29 7.917527934710185
30 7.74956670115071
31 7.587867326289415
32 7.431495859406211
33 7.280724970733418
34 7.135718746185303
35 6.996356868743897
36 6.861390679591411
37 6.731375179792705
38 6.605827775368324
39 6.4844734281301495
40 6.367201862102601
41 6.253897238345373
42 6.144504527712977
43 6.038547579808669
44 5.936222952736749
45 5.8373895466327665
46 5.741735907057499
47 5.6490

368 1.3294504795940265
369 1.32755102784247
370 1.3256677690863288
371 1.3238014053753626
372 1.3219410255671187
373 1.3200842858795176
374 1.3182434156894685
375 1.3164030373572035
376 1.3145803635095095
377 1.312766453229561
378 1.310978920698795
379 1.3091795894817302
380 1.3073979487256429
381 1.305613168996042
382 1.3038440799277382
383 1.3020808181415002
384 1.3003285512676486
385 1.2985968457756882
386 1.296853206308621
387 1.2951322521899165
388 1.2934144372958452
389 1.2916954311040731
390 1.2899874679267864
391 1.288296156756732
392 1.2866043880998936
393 1.28492406166447
394 1.283251966298381
395 1.28159964862797
396 1.2799492489330715
397 1.2783134267258285
398 1.2766790528942769
399 1.275058702737093
400 1.2734326919861267
401 1.271814186715368
402 1.2702041339933428
403 1.2686047454871754
404 1.26700843733034
405 1.2654214490957447
406 1.2638544153522802
407 1.2622790370063455
408 1.2607156725996573
409 1.259162793406626
410 1.2576164207319274
411 1.2560693485499586
412 1

732 0.967443593940094
733 0.9669044144532823
734 0.9663651360845079
735 0.9658291207525231
736 0.9652977487671327
737 0.9647613156516216
738 0.9642258035293774
739 0.9636924107151257
740 0.9631661824510004
741 0.9626317594792805
742 0.9621037321147655
743 0.9615799223203012
744 0.9610536438086689
745 0.9605267702714207
746 0.9600014141983935
747 0.9594792562990105
748 0.9589574901041424
749 0.9584447265525659
750 0.9579213844138519
751 0.9574013375360122
752 0.956887434462548
753 0.9563703549378588
754 0.9558581360029859
755 0.9553438918990276
756 0.9548284726362563
757 0.9543194889216279
758 0.9538117912191805
759 0.9533034413013803
760 0.9527961000800133
761 0.9522916363977542
762 0.9517875206560326
763 0.9512821984365193
764 0.9507747332079738
765 0.9502687910343119
766 0.9497645387651867
767 0.9492656464979519
768 0.9487665719867373
769 0.9482689976053579
770 0.9477755304189198
771 0.947282789244167
772 0.9467896409727932
773 0.9463016710520868
774 0.9458111545712717
775 0.94532026

In [26]:
loss

tensor(0.3657, grad_fn=<MseLossBackward>)

## Test

In [29]:
for batch_number, batch in enumerate(test_data_loader):
        input = batch['input']
        label = batch['label']

        model.eval()
        output = model(input)

        model.zero_grad()

        loss = loss_function(output, label)
        loss_lst.append(loss.item())
print(np.mean(loss_lst))

0.8564694764205023
